<a href="https://colab.research.google.com/github/SamuelaAnastasi/Sequence_to-Sequence_Models/blob/master/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chatbot Project

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
# import libs
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re #regex lib
import os
import unicodedata
import codecs
import itertools

In [0]:
CUDA = torch.cuda.is_available()
device = torch.device('cuda' if CUDA else 'cpu')

In [0]:
CUDA

True

In [0]:
device

device(type='cuda')

In [0]:
# read text files
with open('/gdrive/My Drive/Colab Notebooks/chatbot/cornell_movie_dialogs_corpus/movie_lines.txt', 'rb') as f:
  lines = f.readlines()
  
  for line in lines[:8]:
    print(line.strip())

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go."
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie."
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No'


In [0]:
# split lines in fields and create a dict that contains them
line_fields = ["lineID","characterID","movieID", "character", "text"]
lines = {}

with open('/gdrive/My Drive/Colab Notebooks/chatbot/cornell_movie_dialogs_corpus/movie_lines.txt', 'r', encoding='iso-8859-1') as f:
  for line in f:
    values = line.split(" +++$+++ ")
    
    # extract fields and create a line_object that contain field-value pairs
    lineObj = {}
    
    for i, field in enumerate(line_fields):
      lineObj[field] = values[i]
    lines[lineObj['lineID']] = lineObj

In [0]:
lines

{'L1045': {'character': 'BIANCA',
  'characterID': 'u0',
  'lineID': 'L1045',
  'movieID': 'm0',
  'text': 'They do not!\n'},
 'L1044': {'character': 'CAMERON',
  'characterID': 'u2',
  'lineID': 'L1044',
  'movieID': 'm0',
  'text': 'They do to!\n'},
 'L985': {'character': 'BIANCA',
  'characterID': 'u0',
  'lineID': 'L985',
  'movieID': 'm0',
  'text': 'I hope so.\n'},
 'L984': {'character': 'CAMERON',
  'characterID': 'u2',
  'lineID': 'L984',
  'movieID': 'm0',
  'text': 'She okay?\n'},
 'L925': {'character': 'BIANCA',
  'characterID': 'u0',
  'lineID': 'L925',
  'movieID': 'm0',
  'text': "Let's go.\n"},
 'L924': {'character': 'CAMERON',
  'characterID': 'u2',
  'lineID': 'L924',
  'movieID': 'm0',
  'text': 'Wow\n'},
 'L872': {'character': 'BIANCA',
  'characterID': 'u0',
  'lineID': 'L872',
  'movieID': 'm0',
  'text': "Okay -- you're gonna need to learn how to lie.\n"},
 'L871': {'character': 'CAMERON',
  'characterID': 'u2',
  'lineID': 'L871',
  'movieID': 'm0',
  'text': 'No

In [0]:
list(lines.items())[0]

('L1045',
 {'character': 'BIANCA',
  'characterID': 'u0',
  'lineID': 'L1045',
  'movieID': 'm0',
  'text': 'They do not!\n'})

In [0]:
# split lines in fields and create a dict that contains them
conv_fields = ["character1ID","character2ID","movieID", "utteranceIDs"]
conversations = []

with open('/gdrive/My Drive/Colab Notebooks/chatbot/cornell_movie_dialogs_corpus/movie_conversations.txt', 'r', encoding='iso-8859-1') as f:
  for line in f:
    values = line.split(" +++$+++ ")
    
    # extract fields and create a line_object that contain field-value pairs
    convObj = {}
    
    for i, field in enumerate(conv_fields):
      convObj[field] = values[i]
    
    lineIds = eval(convObj["utteranceIDs"])
    convObj["lines"] = []
    
    for lineId in lineIds:
      convObj["lines"].append(lines[lineId])
    
    conversations.append(convObj)

In [0]:
conversations

[{'character1ID': 'u0',
  'character2ID': 'u2',
  'lines': [{'character': 'BIANCA',
    'characterID': 'u0',
    'lineID': 'L194',
    'movieID': 'm0',
    'text': 'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n'},
   {'character': 'CAMERON',
    'characterID': 'u2',
    'lineID': 'L195',
    'movieID': 'm0',
    'text': "Well, I thought we'd start with pronunciation, if that's okay with you.\n"},
   {'character': 'BIANCA',
    'characterID': 'u0',
    'lineID': 'L196',
    'movieID': 'm0',
    'text': 'Not the hacking and gagging and spitting part.  Please.\n'},
   {'character': 'CAMERON',
    'characterID': 'u2',
    'lineID': 'L197',
    'movieID': 'm0',
    'text': "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"}],
  'movieID': 'm0',
  'utteranceIDs': "['L194', 'L195', 'L196', 'L197']\n"},
 {'character1ID': 'u0',
  'character2ID': 'u2',
  'lines': [{'character': 'B

In [0]:
list(conversations)[0]

{'character1ID': 'u0',
 'character2ID': 'u2',
 'lines': [{'character': 'BIANCA',
   'characterID': 'u0',
   'lineID': 'L194',
   'movieID': 'm0',
   'text': 'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n'},
  {'character': 'CAMERON',
   'characterID': 'u2',
   'lineID': 'L195',
   'movieID': 'm0',
   'text': "Well, I thought we'd start with pronunciation, if that's okay with you.\n"},
  {'character': 'BIANCA',
   'characterID': 'u0',
   'lineID': 'L196',
   'movieID': 'm0',
   'text': 'Not the hacking and gagging and spitting part.  Please.\n'},
  {'character': 'CAMERON',
   'characterID': 'u2',
   'lineID': 'L197',
   'movieID': 'm0',
   'text': "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"}],
 'movieID': 'm0',
 'utteranceIDs': "['L194', 'L195', 'L196', 'L197']\n"}

In [0]:
# Extract pairs of dialogs from the conversations
qa_pairs = []
for conversation in conversations:
  for i in range(len(conversation["lines"]) - 1):
    inputLine = conversation["lines"][i]["text"].strip()
    targetLine = conversation["lines"][i+1]["text"].strip()
    
    if inputLine and targetLine:
      qa_pairs.append([inputLine, targetLine])

In [0]:
len(qa_pairs)

221282

In [0]:
qa_pairs[0]

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you."]

In [0]:
# Create a new text to write qa_pairs
datafile = "/gdrive/My Drive/Colab Notebooks/chatbot/cornell_movie_dialogs_corpus/formatted_movie_lines.txt"
delimiter = '\t'

# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Write new csv file
print("\nWriting newly formatted file---")
with open(datafile, 'w', encoding='utf-8') as outputfile:
  writer = csv.writer(outputfile, delimiter=delimiter)
  for pair in qa_pairs:
    writer.writerow(pair)
    
print("Done writing to file...")


Writing newly formatted file---
Done writing to file...


In [0]:
# Visualize some line from the new file
with open("/gdrive/My Drive/Colab Notebooks/chatbot/cornell_movie_dialogs_corpus/formatted_movie_lines.txt", 'rb') as file:
  lines = file.readlines()

for line in lines[:8]:
  print(line)  

b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\tSeems like she could get a date easy enough...\r\n"
b'Why?\tUnsolved myster

In [0]:
# Create vocabulary class
# assign indeces to tokens
PAD_token = 0 # Used for padding short sentences
SOS_token = 1 # Start-of-sentence-token <START>
EOS_token = 2 # End-of-sentence.token <END>

class Vocabulary:
  def __init__(self, name):
    self.name = name
    self.word2index = {}
    self.word2count = {}
    self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
    self.num_words = 3 # Count, SOS, EOS, PAD
  
  def addSentence(self, sentence):
    for word in sentence.split(' '):
      self.addWord(word)

  def addWord(self, word):
    if word not in self.word2index:
      self.word2index[word] = self.num_words
      self.word2count[word] = 1
      self.index2word[self.num_words] = word
      self.num_words += 1
    else:
      self.word2count[word] +1

  # Remove words below a certain count threshold
  def trim(self, min_count):

    keep_words = []
    for k, v in self.word2count.items():
      if v >= min_count:
        keep_words.append[k]

        print("Keep words {} / {} = {:.4f}".format(len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
          self.addword(word)

In [0]:
# Turn a unicode String to plain ASCII so to remove complex characters and simplify text
def unicodeToAscii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [0]:
unicodeToAscii('Garçon')

'Garcon'

In [0]:
# Clean text: lowercase, trim wh_space, \n, remove non-letter chars
def normalizeString(s):
  s = unicodeToAscii(s.lower().strip())
  
  # Replace any .?! with wh_sp + the char ->Ex.:'!' = ' !'
  # N.B. \1 means: the first bracketed group ->Ex.: [.?!]
  # N.B. r is not to consider \ as a char i.e. r escapes \
  s = re.sub(r"([.!?])", r" \1", s)
  
  # Remove any char that is not a sequence of lower/upper case char
  # N.B.: - means a sequence of chars, + means: 1 or more
  s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
  
  # Replace a sequence of wh_sp by 1 wh_sp - \s is wh_sp
  s = re.sub(r"\s+", r" ", s).strip()
  return s

In [0]:
# Test regex functions
normalizeString("aa123aa!s's     dd?")

'aa aa !s s dd ?'

In [0]:
# Create a new text to write qa_pairs
datafile = "/gdrive/My Drive/Colab Notebooks/chatbot/cornell_movie_dialogs_corpus/formatted_movie_lines.txt"

# Read the file and split it into lines
print("\nReading and proccessing file...")
lines = open(datafile, encoding='utf-8').read().strip().split('\n')

# Split lines into pairs and normalize
pairs = [[normalizeString(s) for s in pair.split('\t')] for pair in lines]
    
print("Done reading...")

voc = Vocabulary('cornell movie dialog corpus')


Reading and proccessing file...
Done reading...


In [0]:
len(pairs)

221282

In [0]:
pairs[0]

['can we make this quick ? roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad . again .',
 'well i thought we d start with pronunciation if that s okay with you .']

In [0]:
# Returns true if both sentences in pair 'p' are under the MAX_LENGTH threshold
MAX_LENGTH = 10 # Max sentence length to consider (max words)

def filterPair(p):
  # Input sequences need to preserve the last word for EOS token
  return len(p[0].split()) < MAX_LENGTH and len(p[1].split()) < MAX_LENGTH

# Filter pairs using filterPair()
def filterPairs(pairs):
  return [pair for pair in pairs if filterPair(pair)]

In [0]:
pairs[1]

['well i thought we d start with pronunciation if that s okay with you .',
 'not the hacking and gagging and spitting part . please .']

In [0]:
pairs = [pair for pair in pairs if len(pair)  > 1] # Remove pairs with len = 1
print("There are {} pairs/conversations in the dataset before filtering".format(len(pairs)))

pairs = filterPairs(pairs)
print("There are {} pairs/conversations in the dataset after filtering".format(len(pairs)))

There are 221282 pairs/conversations in the dataset before filtering
There are 64271 pairs/conversations in the dataset after filtering


In [0]:
# Add in vocabulary question and reply for each pair
for pair in pairs:
  voc.addSentence(pair[0])
  voc.addSentence(pair[1])

print('Counted words:', voc.num_words)
for pair in pairs[:10]:
  print(pair)

Counted words: 18008
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [0]:
# Get rid off words which appear less than a threshold so the model does not get confused
MIN_COUNT = 3

def trimRareWords(voc, pairs, MIN_COUNT):
  # Trim words from the vocabulary that appear less then 3 times
  voc.trim(MIN_COUNT)
  
  # Filter out pairs with trimmed words
  keep_pairs = []
  for pair in pairs:
      input_sentence = pair[0]
      output_sentence = pair[1]
      keep_input = True
      keep_output = True
      
      # Check input_sentence
      for word in input_sentence.split(' '):
        if word not in voc.word2index:
          keep_input = False
          break
          
      # check output sintence
      for word in output_sentence.split(' '):
        if word not in voc.word2index:
          keep_output = False
          break
          
      # Only keep pairs that do not contain trimmed words in the input & output sentence
      if keep_input and keep_output:
        keep_pairs.append(pair)
  
  
  print("Trimmed from {} pairs to {} pairs, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs)/len(pairs)))
  return keep_pairs
    
    
pairs = trimRareWords(voc, pairs, MIN_COUNT)


Trimmed from 64271 pairs to 64271 pairs, 1.0000 of total


In [0]:
# Get a list of words indeces for each sentence
def indexesFromSentence(voc, sentence):
  return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

In [0]:
pairs[1][0]

'you have my word . as a gentleman'

In [0]:
indexesFromSentence(voc, pairs[1][0]) # last index in the output i.e. 2 = EOS_token

[7, 8, 9, 10, 4, 11, 12, 13, 2]

In [0]:
# Define some samples for testing
inp = []
out = []
for pair in pairs[:10]:
  inp.append(pair[0])
  out.append(pair[1])

print(inp)
print(len(inp))
print(out)
print(len(out))

idx_inp = [indexesFromSentence(voc, sentence) for sentence in inp]
idx_out = [indexesFromSentence(voc, sentence) for sentence in out]

print(idx_inp)
print(idx_out)
idx_inp

['there .', 'you have my word . as a gentleman', 'hi .', 'you know chastity ?', 'have fun tonight ?', 'well no . . .', 'then that s all you had to say .', 'but', 'do you listen to this crap ?', 'what good stuff ?']
10
['where ?', 'you re sweet .', 'looks like things worked out tonight huh ?', 'i believe we share an art instructor', 'tons', 'then that s all you had to say .', 'but', 'you always been this selfish ?', 'what crap ?', 'the real you .']
10
[[3, 4, 2], [7, 8, 9, 10, 4, 11, 12, 13, 2], [16, 4, 2], [7, 24, 25, 6, 2], [8, 33, 22, 6, 2], [35, 36, 4, 4, 4, 2], [37, 38, 39, 40, 7, 41, 42, 43, 4, 2], [44, 2], [49, 7, 50, 42, 47, 51, 6, 2], [52, 53, 54, 6, 2]]
[[5, 6, 2], [7, 14, 15, 4, 2], [17, 18, 19, 20, 21, 22, 23, 6, 2], [26, 27, 28, 29, 30, 31, 32, 2], [34, 2], [37, 38, 39, 40, 7, 41, 42, 43, 4, 2], [44, 2], [7, 45, 46, 47, 48, 6, 2], [52, 51, 6, 2], [55, 56, 7, 4, 2]]


[[3, 4, 2],
 [7, 8, 9, 10, 4, 11, 12, 13, 2],
 [16, 4, 2],
 [7, 24, 25, 6, 2],
 [8, 33, 22, 6, 2],
 [35, 36, 4, 4, 4, 2],
 [37, 38, 39, 40, 7, 41, 42, 43, 4, 2],
 [44, 2],
 [49, 7, 50, 42, 47, 51, 6, 2],
 [52, 53, 54, 6, 2]]

In [0]:
# 0 pad fill short sentences
def zeroPadding(l, fillvalue=0):
  return list(itertools.zip_longest(*l, fillvalue=fillvalue))

In [0]:
length = [len(idx) for idx in idx_inp]
max(length)

10

In [0]:
# print how many words are in each sentence for the current batch
length

[3, 9, 3, 5, 5, 6, 10, 2, 8, 5]

In [0]:
# test zerroPadding()
test_result = zeroPadding(idx_inp)
print(len(test_result)) # The max length of sentence is now the num of rows
test_result

10


[(3, 7, 16, 7, 8, 35, 37, 44, 49, 52),
 (4, 8, 4, 24, 33, 36, 38, 2, 7, 53),
 (2, 9, 2, 25, 22, 4, 39, 0, 50, 54),
 (0, 10, 0, 6, 6, 4, 40, 0, 42, 6),
 (0, 4, 0, 2, 2, 4, 7, 0, 47, 2),
 (0, 11, 0, 0, 0, 2, 41, 0, 51, 0),
 (0, 12, 0, 0, 0, 0, 42, 0, 6, 0),
 (0, 13, 0, 0, 0, 0, 43, 0, 2, 0),
 (0, 2, 0, 0, 0, 0, 4, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 2, 0, 0, 0)]

In [0]:
# Method to convert the int index of each word in the sentence into 0 or 1
def binaryMatrix(l, value=0):
  m = []
  for i, seq in enumerate(l):
    
    m.append([])
    
    for token in seq:
      if token == PAD_token:
        
        m[i].append(0)
        
      else:
        m[i].append(1)        
        
  return m

In [0]:
binary_result = binaryMatrix(test_result)
binary_result

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
 [0, 1, 0, 1, 1, 1, 1, 0, 1, 1],
 [0, 1, 0, 1, 1, 1, 1, 0, 1, 1],
 [0, 1, 0, 0, 0, 1, 1, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]

In [0]:
# Returns padded input sentence tensor and as well as a tensor of lengths for each of the sequences in the batch
def inputVar(l, voc):
  
  indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
  lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
  padList = zeroPadding(indexes_batch)
  padVar = torch.LongTensor(padList)
  
  return padVar, lengths  

In [0]:
# Returns padded target sequence tensor, padding mask and max target length
def outputVar(l, voc):
  
  indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
  max_target_len = max([len(indexes) for indexes in indexes_batch])
  padList = zeroPadding(indexes_batch)
  mask = binaryMatrix(padList)
  mask = torch.torch.ByteTensor(mask)
  padVar = torch.LongTensor(padList)
  
  return padVar, mask, max_target_len 
  

In [0]:
# Returns all items for a given pair of batches
def batch2TrainData(voc, pair_batch):
  
  # Sort the questions in descending length
  pair_batch.sort(key=lambda x: len(x[0].split(' ')), reverse=True)
  
  input_batch, output_batch = [], []
  
  for pair in pair_batch:
    input_batch.append(pair[0])
    output_batch.append(pair[1])
  
  inp, lengths = inputVar(input_batch, voc)
  output, mask, max_target_len = outputVar(output_batch, voc)
  
  return inp, lengths, output, mask, max_target_len

In [0]:
# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print('input_variable:')
print(input_variable)
print('lengths: ', lengths)
print('target_variable:')
print(target_variable)
print('mask:')
print(mask)
print('max_target_len: ', max_target_len)

input_variable:
tensor([[   52,   194,   175, 11087,   351],
        [   71,     4,    69,   120,   752],
        [   28,   101,  2972,    12,   658],
        [  699,   931,    69, 11088,    67],
        [  101,     4,   154,     6,     4],
        [  162,     4,   658,     2,     2],
        [  408,     4,     6,     0,     0],
        [    6,     2,     2,     0,     0],
        [    2,     0,     0,     0,     0]])
lengths:  tensor([9, 8, 8, 6, 6])
target_variable:
tensor([[13965,   214,   154,    79,   145],
        [    4,   123,    55,    99,    64],
        [    2,   271,   745,   424,   392],
        [    0,    43,   319,     4,  8894],
        [    0,    79,    26,     2,    42],
        [    0,     4,    24,     0,     7],
        [    0,     2,   154,     0,    78],
        [    0,     0,   658,     0,    86],
        [    0,     0,     6,     0,     6],
        [    0,     0,     2,     0,     2]])
mask:
tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1,

In [0]:
# Define Encoder class
class EncoderRNN(nn.Module):
  def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
    super(EncodderRNN, self).__init__()
    
    self.n_layers = n_layers
    self.hidden_size = hidden_size
    self.embedding = embedding
    
    # Initialize GRU: the input_size and hidden_size params are both set to hidden_size
    # because our input size is a word embeding with num of features = hidden_size
    self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers==1 else dropout), bidirectional=True)
    
    def forward(self, input_seq, input_lengths, hidden=None):
      # input_seq: batch of input sentences - shape = (max_length, batch_size)
      # input_lengths: list of sentence lengths corresponding to each sentence in the batch
      # hidden state of shape: (n_layers x num_directions, batch_size, hidden_size)
      # Convert word indexes to embeddings
      embedded = self.embedding(input_seq)
      
      # Pack padded batch of sequences for RNN module
      packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
      
      # Forward pass through GRU
      outputs, hidden = self.gru(packed, hidden)
      # Unpack padding
      outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
      
      # Sum bidirectional GRU outputs
      outputs = outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:]
      
      # Return output and final hidden_state
      return outputs, hidden
    
    # outputs: the output features h_t from the last layer of GRU, for each time step (sum of bidirectional outputs)
    # outputs shape=(max_length, batch_size, hidden_size)
    # hidden: hidden state for the last timestep of shape=(n_layers x num_directions, batch_size, hidden_size)     
      
    

In [0]:
# Attention mechanism (layer) dot multiplies the encoders output and the decoder's hodden state
class Attn(nn.Module):
  def __init__(self, method, hidden_size):
    super(Attn, self).__init__()
    self.mathod = method
    self.hidden_size = hidden_size
    
    def dot_score(self, hidden, encoder_output):
      # Element wise multiply the the current target decoder state with the encoder output ans sum them
      return torch.sum(hidden * encoder_output, dim=2)
    
    def forward(self, hidden, encoder_outputs):
      # hidden of shape: (1, batch_size, hidden_size)
      # encoder_outputs of shape: (max_length, batch_size, hidden_size) # (max_length, batch_size)
      
      # Calculate the attention weights (energies)
      attn_energies = self.dot_score(hidden, encoder_outputs) # (max_length, batch_size)
      
      # Transpose max_length and batch_size dimensions
      attn_energies = attn_energies.t()  # (batch_size, max_length)
      
      # return the softmax normalized probability scores (with added dimension)
      return F.softmax(attn_energies, dim=1).unsqueeze(1) # (batch_size, 1, max_length)      


In [0]:
# Define Decoder
class LuongAttnDecoderRNN(nn.Module):
  def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
    super(LuongAttnDecoderRNN, self).__init__()
    self.attn_model=attn_model
    self.hidden_size=hidden_size
    self.output_size=output_size
    self.n_layers=n_layers
    self.dropout=dropout
    
    # define layers
    self.embedding=embedding
    self.embedding_dropout=nn.Dropout(dropout)
    
    self.gru=nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers==1 else dropout))
    self.concat = nn.Linear(hidden_size * 2, hidden_size)
    
    self.attn = Attn(attn_model, hidden_size)
    
    def forward(self, input_step, last_hidden, encoder_outputs):
      # input_step: one time step (one word) of input sequence batch; shape=(1, batch_size)
      # last_hidden: final hidden layer of GRU; shape=(n_layers x num_directions, batch_size, hidden_size)
      # encoder_outputs: encoder model's output; shape=(max_length, batch_size, hidden_size)
      # Note: we runthis one step (word) at a time
      
      # get embedding of current input word
      embedded = self.embedding(input_step)
      embedded = self.embedding_dropout(embedded)
      
      # Forward through unidirectional GRU
      rnn_output, hidden = self.gru(embedded, last_hidden)
      # Calculate attention weights from the current GRU output
      attn_weights = self.attn(rnn_output, rnn_outputs)
      
      # Multiply attention_weights to encoder outputs to get new 'weighted sum' context vector
      #(batch_size, 1, max_length) bmm with (batch_size, max_length, hidden) = (batch_size, 1, hidden)
      context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
      
      # Concatenate weighted context vector and GRU output
      rnn_output = rnn_output.squeeze(0)
      context = context.squeeze(1)
      concat_input = torch.cat((rnn_output, context), 1)
      concat_output = torch.tanh(self.concat(concat_input))
      
      # Predict next word
      output = self.out(concat_output)
      output = F.softmax(output, dim=1)
      
      # Return output and final hidden state
      return output, hidden
      # output: softmax normalized vector - Shape: (batch_size, voc.num_words)
      # hidden: final hidden state of GRU - Shape: (n_layers x num_directions, batch_size_hidden_size)  
  

In [0]:
def maskNLLLoss(decoder_out, target, masl):
  nTotal = mask.sum()
  target = traget.view(-1, 1)
  
  # Decoder out shape: (batch_size, vocab_size), target_size = (batch_size, 1)
  gathered_tensor = torch.gather(decoder_out, 1, target)
  
  # Calculate the loss
  crossEntropy = -torch.log(gathered_tensor)
  
  # Select the non-zero elements
  loss = crossEntropy.masked_select(mask)
  
  # Calculate the mean of the loss
  loss = loss.mean()
  loss = loss.to(device)
  
  return loss, nTotal.item()